# **World Championship 2022**
## **49er and 49erFX**

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
options = Options()
options.add_argument("--headless")

Check your chrome version searching for chrome://version. If your chrome version is 115 or newer, you can install chromedriver here https://googlechromelabs.github.io/chrome-for-testing/#stable. Else, check it here https://chromedriver.chromium.org/downloads. After downloading, unzip the file, move it to the same folder as this notebook and inform the path in the next cell.

In [3]:
PATH_TO_CHROMEDRIVER = 'chromedriver-mac-x64/chromedriver'

In [4]:
URL = 'https://49er.org/events/2022-world-championship/#result-49'

driver = Chrome(options=options)
driver.get(URL)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser")

In [5]:
data = []

for row in soup.find_all('tr'):
    resultados = []
    for cell in row.find_all('td'):
        # if cell contains <strike> tag, add () around it
        if cell.find('strike'):
            cell.string = '(' + cell.string + ')'
        resultados.append(cell.text)
    data.append(resultados)

In [6]:
# turn data into a pandas dataframe
df = pd.DataFrame(data)

# change column names
df.columns = ['Posição Geral', 'Sail Number', 'Nome Competidor', 'Nett', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 
              'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Pontuação Total']

# drop Sail Number column
df.drop('Sail Number', axis=1, inplace=True)

In [7]:
df_49er = df.iloc[1:66, :]
df_49erFX = df.iloc[67:103, :]

In [8]:
# turn all the Q1, Q2, ..., Q17 columns into rows
df_49er = pd.melt(df_49er, id_vars=['Posição Geral', 'Nome Competidor', 'Nett', 'Pontuação Total'], 
                  value_vars=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 
                              'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 
                              'Q13', 'Q14', 'Q15', 'Q16', 'Q17'])

df_49erFX = pd.melt(df_49erFX, id_vars=['Posição Geral', 'Nome Competidor', 'Nett', 'Pontuação Total'],
                    value_vars=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 
                                'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 
                                'Q13', 'Q14', 'Q15', 'Q16', 'Q17'])

In [9]:
# rename column 'variable' to 'Flotilha'
df_49er.rename(columns={'variable': 'Flotilha',
                        'value': 'Pontuação Regata'}, inplace=True)

df_49erFX.rename(columns={'variable': 'Flotilha',
                          'value': 'Pontuação Regata'}, inplace=True)

df_49er["Regata"] = df_49er["Flotilha"]
df_49erFX["Regata"] = df_49erFX["Flotilha"]

In [10]:
# define functions to rename fleet names
def rename_fleet_49er(x):
    if x == 'Q17':
        x = 'MR'
    elif x == 'Q16':
        x = 'P'
    else:
        x = 'G'
    return x

def rename_fleet_49erFX(x):
    if x == 'Q16':
        x = 'MR'
    else:
        x = 'G'
    return x

df_49er['Flotilha'] = df_49er['Flotilha'].apply(rename_fleet_49er)
df_49erFX['Flotilha'] = df_49erFX['Flotilha'].apply(rename_fleet_49erFX)

In [11]:
df_49er['Nome Competição'] = ['World Championship 2022'] * len(df_49er)
df_49er['ID Competição'] = [12] * len(df_49er)
df_49er['Classe Vela'] = ['49er'] * len(df_49er)
df_49er['Punição'] = [''] * len(df_49er)

df_49erFX['Nome Competição'] = ['World Championship 2022'] * len(df_49erFX)
df_49erFX['ID Competição'] = [12] * len(df_49erFX)
df_49erFX['Classe Vela'] = ['49erFX'] * len(df_49erFX)
df_49erFX['Punição'] = [''] * len(df_49erFX)

In [12]:
column_order = ['Nome Competidor', 'ID Competição', 'Classe Vela', 
                'Pontuação Regata', 'Flotilha', 'Posição Geral', 
                'Punição', 'Pontuação Total', 'Nett', 'Nome Competição', 'Regata']

df_49er = df_49er[column_order]
df_49erFX = df_49erFX[column_order]

In [13]:
df_49er.head()

,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata
0,Bart LAMBRIEXFloris van de WERKEN,12,49er,12.0,G,1,,100.0,85.0,World Championship 2022,Q1
1,Diego BOTINFlorian TRITTEL PAUL,12,49er,6.0,G,2,,132.0,107.0,World Championship 2022,Q1
2,Sime FANTELAMihovil FANTELA,12,49er,3.0,G,3,,134.0,109.0,World Championship 2022,Q1
3,Isaac McHARDIEWilliam McKENZIE,12,49er,(22.0),G,4,,138.0,116.0,World Championship 2022,Q1
4,Łukasz PRZYBYTEKJacek PIASECKI,12,49er,3.0,G,5,,138.0,117.0,World Championship 2022,Q1


In [14]:
df_49erFX.head()

,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata
0,Odile van AANHOLTAnnette DUETZ,12,49erFX,2.0,G,1,,113.0,84.0,World Championship 2022,Q1
1,Vilma BOBECKRebecca NETZLER,12,49erFX,5.0,G,2,,134.0,103.0,World Championship 2022,Q1
2,Támara ECHEGOYENPaula BARCELÓ,12,49erFX,4.0,G,3,,151.0,127.0,World Championship 2022,Q1
3,Aleksandra MELZACKASandra JANKOWIAK,12,49erFX,17.0,G,4,,183.0,151.0,World Championship 2022,Q1
4,Stephanie ROBLEMaggie SHEA,12,49erFX,14.0,G,5,,187.0,155.0,World Championship 2022,Q1


In [15]:
df_49er.to_csv('../scraped-data/new_World Championship 2022_49er.csv', index=False)
df_49erFX.to_csv('../scraped-data/new_World Championship 2022_49erFX.csv', index=False)

---

In [16]:
URL = 'https://nacra17.org/event/2022-world-championship/#nacra17-results'

driver = Chrome(options=options)
driver.get(URL)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser")

In [17]:
data = []

for row in soup.find_all('tr'):
    resultados = []
    for cell in row.find_all('td'):
        # if cell contains <strike> tag, add () around it
        if cell.find('strike'):
            cell.string = '(' + cell.string + ')'
        resultados.append(cell.text)
    data.append(resultados)

In [18]:
# turn data into a pandas dataframe
df = pd.DataFrame(data)

# change column names
df.columns = ['Posição Geral', 'Sail Number', 'Nome Competidor', 'Nett', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 
              'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Pontuação Total']

# drop Sail Number column
df.drop('Sail Number', axis=1, inplace=True)

In [19]:
# turn all the Q1, Q2, ..., Q17 columns into rows
df = pd.melt(df, id_vars=['Posição Geral', 'Nome Competidor', 'Nett', 'Pontuação Total'], 
             value_vars=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 
                         'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 
                         'Q13', 'Q14', 'Q15', 'Q16', 'Q17'])

In [20]:
# rename column 'variable' to 'Flotilha'
df.rename(columns={'variable': 'Flotilha',
                   'value': 'Pontuação Regata'}, inplace=True)

df["Regata"] = df["Flotilha"]

In [21]:
def rename_fleet_49erFX(x):
    if x == 'Q17':
        x = 'MR'
    else:
        x = 'G'
    return x

df['Flotilha'] = df['Flotilha'].apply(rename_fleet_49erFX)

In [22]:
df['Nome Competição'] = ['World Championship 2022'] * len(df)
df['ID Competição'] = [12] * len(df)
df['Classe Vela'] = ['Nacra17'] * len(df)
df['Punição'] = [''] * len(df)

In [23]:
df.head()

,Posição Geral,Nome Competidor,Nett,Pontuação Total,Flotilha,Pontuação Regata,Regata,Nome Competição,ID Competição,Classe Vela,Punição
0,None,None,None,None,G,None,Q1,World Championship 2022,12,Nacra17,
1,1,Ruggero TITACaterina BANTI,26.0,46.0,G,1.0,Q1,World Championship 2022,12,Nacra17,
2,2,Gianluigi UGOLINIMaria GIUBILEI,86.0,115.0,G,3.0,Q1,World Championship 2022,12,Nacra17,
3,3,Sinem KURTBAYAkseli KESKINEN,101.0,115.0,G,5.0,Q1,World Championship 2022,12,Nacra17,
4,4,John GIMSONAnna BURNET,104.0,122.0,G,4.0,Q1,World Championship 2022,12,Nacra17,


In [24]:
df.to_csv('../scraped-data/new_World Championship 2022_Nacra17.csv', index=False)